# Usando o Spark para fazer um breve EDA e depois fazer um Undersampling

- Chamando o PySpark e criando uma sessão

In [2]:
import warnings
warnings.filterwarnings('ignore')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MyApp").getOrCreate()

- Lendo o dataset utilizando o spark

In [3]:
path = '1. Dados/fraud_detection_dataset.csv'

df = spark.read.csv(path,
                    inferSchema="true", #spark precisa disso para ler os tipos dos dados
                    header=True #spark precisa disso para ler o nome das colunas
                   ) 
df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

- Analisando o tipo dos dados e quantidade de nulos, similar ao pandas.info(

In [3]:
df.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)



- Similar ao pandas.describe()

In [4]:
df.summary().show()

+-------+------------------+--------+------------------+-----------+------------------+-----------------+-----------+------------------+------------------+--------------------+--------------------+
|summary|              step|    type|            amount|   nameOrig|     oldbalanceOrg|   newbalanceOrig|   nameDest|    oldbalanceDest|    newbalanceDest|             isFraud|      isFlaggedFraud|
+-------+------------------+--------+------------------+-----------+------------------+-----------------+-----------+------------------+------------------+--------------------+--------------------+
|  count|           6362620| 6362620|           6362620|    6362620|           6362620|          6362620|    6362620|           6362620|           6362620|             6362620|             6362620|
|   mean|243.39724563151657|    null|179861.90354913048|       null| 833883.1040744851|855113.6685785787|       null|1100701.6665196505|1224996.3982019336|0.001290820448180152| 2.51468734577894E-6|
| stddev| 

- Agrupando por isFraud para ver os dados

In [4]:
df.groupBy('isFraud').avg().show()

+-------+------------------+------------------+------------------+-------------------+-------------------+-------------------+------------+--------------------+
|isFraud|         avg(step)|       avg(amount)|avg(oldbalanceOrg)|avg(newbalanceOrig)|avg(oldbalanceDest)|avg(newbalanceDest)|avg(isFraud)| avg(isFlaggedFraud)|
+-------+------------------+------------------+------------------+-------------------+-------------------+-------------------+------------+--------------------+
|      1| 368.4138560818215|1467967.2991403877|1649667.6057116778| 192392.63183611355|  544249.6190746377| 1279707.6171459886|         1.0|0.001948131011810...|
|      0|243.23566306029815| 178197.0417274069| 832828.7117272614|  855970.2281088016| 1101420.8745693814| 1224925.6845631644|         0.0|                 0.0|
+-------+------------------+------------------+------------------+-------------------+-------------------+-------------------+------------+--------------------+



- contabilizando o ratio entre isFraud = 0 e isFraud = 1 para iniciar o Undersampling 

In [20]:
major_df = df.filter("isFraud==0")
minor_df = df.filter("isFraud== 1")
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))

ratio: 773


- Criando um dataset com fraudes 50% positivas e 50% negativas

In [21]:
sampled_majority_df = major_df.sample(False, 1/ratio)
combined_df_2 = sampled_majority_df.unionAll(minor_df)
combined_df_2.show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| CASH_IN| 62871.06| C735228558|   8367720.25|     8430591.3| C716157500|     177707.91|       4894.45|      0|             0|
|   1| CASH_IN|217668.38| C662925691|   1252326.68|    1469995.06| C476800120|     347373.26|      49864.36|      0|             0|
|   1|TRANSFER| 127920.4| C557197074|          0.0|           0.0| C977993101|     330781.11|     965870.05|      0|             0|
|   1| PAYMENT|  6433.58|C1831875736|    122027.28|      115593.7|M1806121021|           0.0|           0.0|      0|             0|
|   2|TRANSFER|308871.37|C1074976188|      31486.0|           0.0|C185004209

- Visualizando a descrição do novo dataset

In [22]:
combined_df_2.summary().show()

+-------+------------------+--------+-----------------+-----------+------------------+-----------------+-----------+------------------+------------------+------------------+--------------------+
|summary|              step|    type|           amount|   nameOrig|     oldbalanceOrg|   newbalanceOrig|   nameDest|    oldbalanceDest|    newbalanceDest|           isFraud|      isFlaggedFraud|
+-------+------------------+--------+-----------------+-----------+------------------+-----------------+-----------+------------------+------------------+------------------+--------------------+
|  count|             16322|   16322|            16322|      16322|             16322|            16322|      16322|             16322|             16322|             16322|               16322|
|   mean| 305.4622595270188|    null|824582.7800140918|       null| 1247924.033909448|526349.3524653842|       null| 820527.5379775766|  1246544.02767553|0.5031858840828329|9.802720254870727E-4|
| stddev|193.675945189653

- Criando um novo arquivo .csv com o undersampling dataset

In [23]:
combined_df_2.toPandas().to_csv('balanceado.csv')